In [1]:
import typing as t

import torch
import torch.nn as nn

# Regression

In [2]:
class Regression(nn.Module):

    def __init__(self, num_features: int):
        super().__init__()
        self.mlp = nn.Sequential(  # многослойный перцептрон?
            # слой 1
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            # слой 2
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.25),
            # слой 3
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            # выход
            nn.Linear(512, 1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.mlp(x)

# Feature classification

## Simple

In [3]:
class FeatureClassifier(nn.Module):

    def __init__(self, num_features: int, num_classes: int):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.classifier(x)

## CNN

In [4]:
class FeatureCNNClassifier(nn.Module):
    LAST_CONV_OUT_CHANNELS = 64
    ADAPTIVE_AVG_POOL = 16

    def __init__(self, num_classes: int):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=self.LAST_CONV_OUT_CHANNELS, kernel_size=2),
            nn.BatchNorm1d(num_features=self.LAST_CONV_OUT_CHANNELS),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=2),
        )
        self.avgpool = nn.AdaptiveAvgPool1d(self.ADAPTIVE_AVG_POOL)
        self.classifier = nn.Sequential(
            nn.Linear(self.LAST_CONV_OUT_CHANNELS * self.ADAPTIVE_AVG_POOL, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x.unsqueeze(1))
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

## RNN

In [5]:
class FeatureRNNClassifier(nn.Module):
    _STATE_T = t.Union[t.Optional[torch.Tensor], t.Optional[t.Tuple[torch.Tensor, torch.Tensor]]]
    rnn_state: _STATE_T

    def __init__(self, num_features: int, hidden_size: int, num_classes: int):
        super().__init__()
        self.rnn = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_size,
            num_layers=2,
            dropout=0.25,
            batch_first=True,
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, num_classes),
        )
        self.reset_rnn_state()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x, rnn_state = self.rnn(x, self.rnn_state)
        self.keep_rnn_state(rnn_state)

        x = torch.flatten(x, 1)
        return self.classifier(x)

    def reset_rnn_state(self):
        self.rnn_state = None

    def keep_rnn_state(self, state: _STATE_T):
        if isinstance(self.rnn, nn.LSTM):
            self.rnn_state = (state[0].detach(), state[1].detach())
        else:
            self.rnn_state = state.detach()

    def train(self, mode: bool = True):
        self.reset_rnn_state()
        return super().train(mode)

# Image classification

## Simple

## CNN

In [6]:
class ImageCNNClassifier(nn.Module):
    LAST_CONV_OUT_CHANNELS = 64
    ADAPTIVE_AVG_POOL = 4

    def __init__(self, num_channels: int, num_classes: int):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=num_channels, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=32, out_channels=self.LAST_CONV_OUT_CHANNELS, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(self.LAST_CONV_OUT_CHANNELS),
            nn.ReLU(True),
            nn.MaxPool2d(2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((self.ADAPTIVE_AVG_POOL, self.ADAPTIVE_AVG_POOL))
        self.classifier = nn.Linear(
            self.LAST_CONV_OUT_CHANNELS * self.ADAPTIVE_AVG_POOL * self.ADAPTIVE_AVG_POOL,
            num_classes,
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

## RNN

# Text classification

## Simple

In [7]:
class TextClassifier(nn.Module):

    def __init__(
            self,
            num_embeddings: int,  # размер алфавита
            embedding_dim: int,
            vector_size: int,  # максимальная длина текста
            num_classes: int,
    ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        self.classifier = nn.Sequential(  # можно менять кол-во слоев
            nn.Linear(embedding_dim * vector_size, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(64, 128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(64, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

## CNN

In [8]:
class TextCNNClassifier(nn.Module):
    LAST_CONV_OUT_CHANNELS = 64
    ADAPTIVE_AVG_POOL = 8

    def __init__(
            self,
            num_embeddings: int,  # размер алфавита
            embedding_dim: int,
            num_classes: int,
    ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        self.features = nn.Sequential(  # можно менять кол-во слоев
            # полный слой 1
            nn.Conv1d(in_channels=embedding_dim, out_channels=32, kernel_size=2),
            nn.BatchNorm1d(num_features=32),  # опционально
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            # полный слой 1
            nn.Conv1d(in_channels=32, out_channels=self.LAST_CONV_OUT_CHANNELS, kernel_size=2),
            nn.BatchNorm1d(num_features=self.LAST_CONV_OUT_CHANNELS),  # опционально
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
        )
        self.avgpool = nn.AdaptiveAvgPool1d(self.ADAPTIVE_AVG_POOL)
        self.classifier = nn.Sequential(  # можно менять кол-во слоев
            nn.Linear(self.LAST_CONV_OUT_CHANNELS * self.ADAPTIVE_AVG_POOL, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)
        x = torch.permute(x, dims=(0, 2, 1))

        x = self.features(x)
        x = self.avgpool(x)

        x = torch.flatten(x, 1)
        return self.classifier(x)

## RNN

In [9]:
class TextRNNClassifier(nn.Module):
    _STATE_T = t.Union[t.Optional[torch.Tensor], t.Optional[t.Tuple[torch.Tensor, torch.Tensor]]]
    rnn_state: _STATE_T

    def __init__(
            self,
            num_embeddings: int,  # размер алфавита
            embedding_dim: int,
            rnn_hidden_size: int,
            vector_size: int,  # максимальная длина текста
            num_classes: int,
    ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        # любой из nn.RNN, nn.LSTM, nn.GRU
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=rnn_hidden_size, batch_first=True)
        self.classifier = nn.Sequential(  # можно менять кол-во слоев
            nn.Linear(rnn_hidden_size * vector_size, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, num_classes),
        )
        self.reset_rnn_state()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)

        x, rnn_state = self.rnn(x, self.rnn_state)
        self.keep_rnn_state(rnn_state)

        x = torch.flatten(x, 1)
        return self.classifier(x)

    def reset_rnn_state(self):
        self.rnn_state = None

    def keep_rnn_state(self, state: _STATE_T):
        if isinstance(self.rnn, nn.LSTM):
            self.rnn_state = (state[0].detach(), state[1].detach())
        else:
            self.rnn_state = state.detach()

    def train(self, mode: bool = True):
        self.reset_rnn_state()
        return super().train(mode)